# Components

> Componenents that are the building blocks to the UI

In [1]:
#| default_exp components

In [2]:
#| hide
#| export
from fasthtml.common import *
from fasthtml.svg import Svg
from enum import Enum, EnumType
from fasthtml.components import Uk_select,Uk_input_tag
from functools import partial


## Utils

### Tests

In [3]:
from fastcore.test import *

In [4]:
def test_xml_eq(a, b):
    test_eq(*map(lambda x: x if isinstance(x,str) else to_xml(x), (a, b)))

## Other

In [5]:
#| export
def stringify(o):
    # need a better name, stringify might be too general for what it does 
    if is_listy(o): return ' '.join(map(str,o)) if o else ""
    return o.__str__()

In [6]:
assert stringify('abc') == 'abc'
assert stringify(('abc','def')) == 'abc def'
assert 'uk-input ' + stringify(()) == 'uk-input '
assert 'uk-input ' + stringify("") == 'uk-input '

### Components

In [7]:
#| export
class VEnum(Enum):
    def __add__(self, other):
        return self.__str__() + other.__str__()
    
    def __str__(self):
        base = self.__class__.__name__       
        if isinstance(self.__class__, EnumType):
            base = base.lstrip('Uk').rstrip('T')
        return f"uk-{base.lower()}-{self.value}".strip('-')

In [8]:
# class UkTextT(VEnum):
#     red = 'red'
#     blue = 'blue'
#     small = 'small'
#     large = 'large'
    

# test_xml_eq(Div(cls=UkTextT.red),'<div class="uk-text-red"></div>\n')
# test_xml_eq(Div(cls=(UkTextT.red,UkTextT.small)),'<div class="uk-text-red uk-text-small"></div>\n')

In [9]:
#| export
class TextB(Enum):
    xsmall = 'text-xs'
    small = 'text-sm'
    muted = 'uk-text-muted'

    def __str__(self):
        return self.value

In [10]:
#| export
class TextT(Enum):
    muted_sm = TextB.small, TextB.muted # Text below card headings

    def __str__(self):
        if is_listy(self.value): return ' '.join(map(str,self.value))
        return self.value

str(TextT.muted_sm)

'text-sm uk-text-muted'

## Themes

Choose a theme for your app.

In [11]:
#| export
class Theme(Enum):
    slate = "slate"
    stone = "stone"
    gray = "gray"
    neutral = "neutral"
    red = "red"
    rose = "rose"
    orange = "orange"
    green = "green"
    blue = "blue"
    yellow = "yellow"
    violet = "violet"
    zinc = "zinc"

    def headers(self):
        js = (Script(src="https://cdn.tailwindcss.com"),
              Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"),
              Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"),
              Script(type="module", src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js")
              )
        _url = "https://unpkg.com/franken-wc@0.0.6/dist/css/{theme}.min.css"
        return (*js, Link(rel="stylesheet", href=_url.format(theme=self.value)))



In [12]:
hdrs = Theme.blue.headers()

## Base Components

### Inputs

In [65]:
#| export
def UkGenericInput(input_fn,
                    label=(), 
                    lbl_cls=(),
                    inp_cls=(),
                    cls=('space-y-2',), # Div cls
                    id="", **kwargs):
    lbl_cls, inp_cls, cls = map(stringify,(lbl_cls, inp_cls, cls))
    if label: 
        label = Label(cls='uk-form-label '+lbl_cls)(label)
        if id: label.fr = id
    res = input_fn(**kwargs)
    if inp_cls: res.attrs['class'] += inp_cls
    if id: res.id = id
    return Div(cls=cls)(label, res)

In [67]:
#| export
UkInput =     partial(UkGenericInput, partial(Input,        cls='uk-input'))
UkSwitch =    partial(UkGenericInput, partial(Input,        cls='uk-toggle-switch uk-toggle-switch-primary', type='checkbox'))
UkTextArea =  partial(UkGenericInput, partial(Textarea,     cls='uk-textarea'))
UkFormLabel = partial(UkGenericInput, partial(Uk_input_tag, cls='uk-form-label'))
UkSelect =    partial(UkGenericInput, partial(Uk_select,                          uk_cloak = True))

In [19]:
team_members = [
    {"name": "Olivia Martin", "email": "m@example.com", "role": "Read and write access"},
    {"name": "Isabella Nguyen", "email": "b@example.com", "role": "Read-only access"},
    {"name": "Sofia Davis", "email": "p@example.com", "role": "Read-only access"},]

options = ("Read and write access", "Read-only access")

member = team_members[0]
print(to_xml(UkSelect(*[Option(o) for o in options], selected=member['role'])))


<div class="space-y-2"><uk-select uk-cloak selected="Read and write access" class="">
  <option>Read and write access</option>
  <option>Read-only access</option>
</uk-select>
</div>

<div class="space-y-2"><uk-select uk-cloak selected="Read and write access" class="">
  <option>Read and write access</option>
  <option>Read-only access</option>
</uk-select>
</div>



In [ ]:
team_members = [
    {"name": "Sofia Davis", "email": "m@example.com", "role": "Owner"},
    {"name": "Jackson Lee", "email": "p@example.com", "role": "Member"},
]

In [ ]:
options = (
        (Div('Viewer'),Div(cls='text-sm text-muted-foreground')('Can view and comment.',)),
        (Div('Developer'),Div(cls='text-sm text-muted-foreground')('Can view, comment and edit.',)),
        (Div('Billing'),Div(cls='text-sm text-muted-foreground')('Can view, comment and manage billing.',)),
        (Div('Owner'),Div(cls='text-sm text-muted-foreground')('Admin-level to all resources.')),
        )
options

((div(('Viewer',),{}),
  div(('Can view and comment.',),{'class': 'text-sm text-muted-foreground'})),
 (div(('Developer',),{}),
  div(('Can view, comment and edit.',),{'class': 'text-sm text-muted-foreground'})),
 (div(('Billing',),{}),
  div(('Can view, comment and manage billing.',),{'class': 'text-sm text-muted-foreground'})),
 (div(('Owner',),{}),
  div(('Admin-level to all resources.',),{'class': 'text-sm text-muted-foreground'})))

In [ ]:
#| export
def UkDropdownButton(label, # Shown on the button
                     options, # list of tuples that contain what you want listed
                     btn_cls=UkButtonT.default, # Button class
                     cls=() # parent div class
                     ):
    btn_cls, cls = map(stringify,(btn_cls, cls))
    btn = Button(type='button', cls='uk-button ' + btn_cls)(label, Span(uk_icon='icon: triangle-down'))
    dd_opts = [Li(A(href="#demo", cls='uk-drop-close',uk_toggle=True, role="button")(Div(o))) for o in options]
    dd = Div(uk_drop='mode: click; pos: bottom-right', cls='uk-dropdown uk-drop')(Ul(cls='uk-dropdown-nav')(*([Li(cls='uk-nav-divider')] + dd_opts)))
    return Div(cls=cls)(Div(cls='flex items-center space-x-4')(btn, dd))

### Other

In [20]:
#| export
class UkButtonT(VEnum):
    default = 'default'
    primary = 'primary'
    secondary = 'secondary'
    danger = 'danger'
    ghost = 'ghost'
    text = 'text'
    link = 'link'

In [81]:
#| export
def UkButton(*c, 
            cls=UkButtonT.default, # Use UkButtonT or styles 
            **kwargs):    
    return Button(cls='uk-button ' + stringify(cls), **kwargs)(*c)


In [83]:
#| export
def UkGenericComponent(component_fn, *c, cls=(), **kwargs):
        res = component_fn(**kwargs)(*c)
        if cls: res.attrs['class'] += ' ' + cls
        return res

UkH1 = partial(UkGenericComponent, partial(H1,cls='uk-h1'))
UkH2 = partial(UkGenericComponent, partial(H2,cls='uk-h2'))
UkH3 = partial(UkGenericComponent, partial(H3,cls='uk-h3'))
UkH4 = partial(UkGenericComponent, partial(H4,cls='uk-h4'))
UkH5 = partial(UkGenericComponent, partial(H5,cls='uk-h5'))
UkH6 = partial(UkGenericComponent, partial(H6,cls='uk-h6'))


In [86]:
UkH1("My Uk styled header"), UkH1('My Uk styled header',cls='another-class')

(h1(('My Uk styled header',),{'class': 'uk-h1'}),
 h1(('My Uk styled header',),{'class': 'uk-h1 another-class'}))

In [68]:
#| export
def UkHSplit(*c, cls=(), line_cls=(), text_cls=()):
    cls, line_cls, text_cls = map(stringify,(cls, line_cls, text_cls))
    return Div(cls='relative ' + cls)(
        Div(cls="absolute inset-0 flex items-center " + line_cls)(Span(cls="w-full border-t border-border")),
        Div(cls="relative flex justify-center " + text_cls)(Span(cls="bg-background px-2 ")(*c)))

## Higher order components

In [ ]:
#|export
def Card(*c, # Components that go in the body
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls=(), # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [Div(cls='uk-card-header ' + header_cls)(header),]
    res += [Div(cls='uk-card-body ' + body_cls)(*c),]
    if footer: res += [Div(cls='uk-card-footer ' + footer_cls)(footer),]
    return Div(cls='uk-card '+cls, **kwargs)(*res)

In [ ]:
#| hide


import nbdev; nbdev.nbdev_export()